In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/tabu

/content/drive/MyDrive/tabu


In [9]:
import numpy as np
import copy
import random 
import os
import time
import json

# Input

In [12]:
def loadData(path):
    f = open(path, "r")
    data = json.load(f)

    N, m, M, d, s, e = data["N"], data["m"], data["M"], data["d"], data["s"], data["e"]
    date = np.max(e)+1
    # print(d)
    # print(s)
    # print(e)
    # print(date)
    return  N, m, M, d, s, e, date

In [32]:
N, m, M, d, s, e, date = loadData('sample_50_50_100.json')
seed = 5

# Cost function


In [26]:
def getCost(N, m, M, d, date, sol):
    amount = np.zeros(date)
    penalty = 0
    for i in range(N):
        amount[sol[i]] += int(d[i])
    for i in amount:
        if (i > M):
            penalty += i-M
        elif (i > 0 and i < m):
            penalty += m-i
    amount_temp = np.delete(amount, amount == 0)
    cost = np.max(amount_temp)-np.min(amount_temp)
    return int(cost+1000*penalty)

# Init

In [16]:
def initialSolution(N, m, M, d, s, e, date, SEED):
    random.seed(SEED)
    sol = [0]*N
    amount = np.zeros(date)
    global flag
    flag = [0]*date

    # cánh đồng nào thu nhiều thì ưu tiên chọn trước
    order = np.argsort(-np.array(d))
    for i in order:
        temp1 = [j for j in range(s[i], e[i]+1) if flag[j] <= 1]
        temp2 = [j for j in range(s[i], e[i]+1) if (flag[j] == 2 and amount[j] + d[i] <= M)]
        if (len(temp1) != 0):
            day = random.choice(temp1)
        elif (len(temp2) != 0):
            day = random.choice(temp2)
        else:
            day = random.randint(s[i], e[i])

        sol[i] = day
        amount[day] += d[i]
        if (amount[day] > M):
            flag[day] = 3  # đã đến giới hạn
        elif (amount[day] >= m):
            flag[day] = 2  # vừa đủ
        elif (amount[day] > 0):
            flag[day] = 1  # chưa đủ         
    return sol

In [ ]:
initialSolution(N, m, M, d, s, e, date, SEED=seed)

# Tabu Search

## Search for next solution

In [18]:
def search_next_solution(sol,SEED):    
    random.seed(SEED)   
    select_field = -1
    select_day = -1
    new_search_objective = 1e9
    global flag
    flag = [0]*date

    amount = np.zeros(date)
    penalty =0
    for i in range(N):
        amount[sol[i]]+=d[i]
    for i in range(date):    
        penalty+=max(amount[i]-M,0)+max(0,min(amount[i],m-amount[i]))

        if (amount[i]>M):
            flag[i]=3 #đã đến giới hạn
        elif (amount[i]>=m):
            flag[i]=2
        elif (amount[i]>0):
            flag[i]=1
    # cost = np.max(amount)-np.min(amount)+1000*penalty
    
    randomOrder = np.arange(N)
    random.shuffle( randomOrder)
    for i in  randomOrder:
        if tabuList[i] > 0:
            continue             
        for j in range(s[i],e[i]+1):
            if (j == sol[i] or flag[j]==0 or flag[j]==3):
                continue    
            max(i-M,0)+max(0,min(i,m-i))
            oldPenalty = max(amount[sol[i]]-M,0)+max(0,min(amount[sol[i]],m-amount[sol[i]])) + max(amount[j]-M,0)+max(0,min(amount[j],m-amount[j]))
            amount[sol[i]]-=d[i]
            amount[j]+=d[i]
            newPenalty = max(amount[sol[i]]-M,0)+max(0,min(amount[sol[i]],m-amount[sol[i]])) + max(amount[j]-M,0)+max(0,min(amount[j],m-amount[j]))
            newSolObjective = np.max(amount)-np.min(amount) + 1000*(penalty-oldPenalty+newPenalty)
            #restore
            amount[sol[i]]+=d[i]
            amount[j]-=d[i]

            if newSolObjective < new_search_objective:
                new_search_objective = newSolObjective
                select_field = i
                select_day = j

    return select_field, select_day, new_search_objective

## TabuSearch Algorithm

In [35]:
def TabuSearch(outputPath,Name,SEED):   
    solve = initialSolution(N, m, M, d, s, e, date, SEED)
    bestSolve = np.array(solve)
    
    global tabuList
    itmax = 300
    it=0
    tabuList = np.zeros(N)
    tbl = 7
    tbl_min = 5
    tbl_max = 9
    stable = 0
    stable_limit = 10
    restart_freq = 25

    f =  open(outputPath,"w")
    f.write("Name: "+Name+"\n")

    f.write("Gen{}: {} {}\n".format(it,getCost(N, m, M, d, date, solve),getCost(N, m, M, d, date, bestSolve)))

    while (it < itmax):
        it+=1

        #Move
        oldSolve = np.array(solve)
        old_search_objective = getCost(N, m, M, d, date, solve)
        field, day, new_search_objective = search_next_solution(solve,SEED+it)
        solve[field] = day
        search_objective = new_search_objective

        #Update tabu
        for i in range(N):
            if tabuList[i] > 0:
                tabuList[i] -= 1
        tabuList[field] = tbl

        if new_search_objective < old_search_objective:
            if tbl > tbl_min:
                tbl -= 1
        else:
            if tbl < tbl_max:
                tbl += 1

        #Update last improved solution
        if new_search_objective < old_search_objective:
            # print("Improved")
            stable = 0
        else:
            stable += 1

        #Cập nhật lời giải kỉ lục
        if new_search_objective < getCost(N, m, M, d, date, bestSolve):
            bestSolve = np.array(solve)
        elif stable == stable_limit:   
            # print("Jump")
            # solve = np.array(lastImproveSolve) 
            stable = 0
            # Chuyển sang không gian khác
            solve = initialSolution(N, m, M, d, s, e, date, SEED+it)
            #Update tabu
            tabuList = np.zeros(N)
          
        # print("Gen{}: {} {}".format(it,getCost(solve),getCost(bestSolve)))
        # show(bestSolve) 
        f.write("Gen{}: {} {}\n".format(it,getCost(N, m, M, d, date, solve),getCost(N, m, M, d, date, bestSolve)))
    f.close()
    return bestSolve

In [36]:
TabuSearch('123.txt', '50_50_100',SEED = 10)

array([ 1,  1,  0,  0,  1,  1,  3,  0,  0,  2,  1,  0,  3,  3,  2,  5,  2,
        2,  4,  6,  5,  4,  7,  2,  4,  6,  2,  4,  7,  7,  5,  6,  3,  4,
        4,  7,  5, 10, 10,  8,  8,  6,  6,  9,  8, 10,  7,  9, 10,  9])

In [ ]:
[1, 1, 0, 1, 0, 0, 0, 1, 1, 0]

In [38]:
getCost(N, m, M, d, date, np.array([ 1,  1,  0,  0,  1,  1,  3,  0,  0,  2,  1,  0,  3,  3,  2,  5,  2,
        2,  4,  6,  5,  4,  7,  2,  4,  6,  2,  4,  7,  7,  5,  6,  3,  4,
        4,  7,  5, 10, 10,  8,  8,  6,  6,  9,  8, 10,  7,  9, 10,  9]))

7

#Output log

In [ ]:
def logOutput(outputPath,Name,time,SEED,sol):
    f = open(outputPath,"w")
    f.write("Name: "+Name+"\n")
    f.write("Time: {} \n".format(time))
    f.write("Result: {}\n".format(getCost(sol)))
    f.write("Solution: ")
    for i in sol:
        f.write(" {}".format(i))
    f.write("\n")
    amount = np.zeros(date)
    day = [[] for i in range(date)]
    for i in range(N):
        amount[sol[i]]+=d[i]
        day[sol[i]].append(i)

    for i in range(date):
        f.write("{} : {} ({})\n".format(i,day[i],amount[i]))
        # print("----sum: {}".format(amount[i]))